We initialize Python imports and opens a DuckDB connection that every later cell reuses

In [ ]:
from pathlib import Path
import duckdb
import numpy as np
import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf

DB_DIR = Path("../db")
DB_DIR.mkdir(parents=True, exist_ok=True)

con = duckdb.connect(str(DB_DIR / "nflpa.duckdb"))

PANEL_TABLE = "team_week_panel"
MODEL_VIEW = "team_week_panel_nextweek_model"

print("db file", (DB_DIR / "nflpa.duckdb").resolve())

existing_views = set(con.execute("SHOW TABLES").df()["name"].tolist())
if MODEL_VIEW not in existing_views:
    raise RuntimeError("Missing model view team_week_panel_nextweek_model, run notebook 11 before step 13")

cols = con.execute("DESCRIBE team_week_panel").df()["column_name"].tolist()
cols_set = set(cols)

if "team_key" in cols_set:
    TEAM_COL = "team_key"
elif "team" in cols_set:
    TEAM_COL = "team"
else:
    raise RuntimeError("Could not find team column in team_week_panel, expected team_key or team")

con.execute("DROP VIEW IF EXISTS team_week_panel_nextweek_model")
con.execute("DROP TABLE IF EXISTS panel_next_week_flags")

con.execute(f"""
CREATE TABLE panel_next_week_flags AS
WITH base AS (
  SELECT
    season,
    week,
    {TEAM_COL} AS team_key
  FROM team_week_panel
)
SELECT
  season,
  week,
  team_key,
  CASE
    WHEN EXISTS (
      SELECT 1
      FROM base b2
      WHERE b2.season = b1.season
        AND b2.team_key = b1.team_key
        AND b2.week = b1.week + 1
    )
    THEN 1
    ELSE 0
  END AS has_next_week
FROM base b1
""")

con.execute(f"""
CREATE VIEW team_week_panel_nextweek_model AS
SELECT
  p.*,
  f.has_next_week
FROM team_week_panel p
JOIN panel_next_week_flags f
  ON p.season = f.season
 AND p.week = f.week
 AND p.{TEAM_COL} = f.team_key
WHERE f.has_next_week = 1
""")

con.execute("SELECT COUNT(*) AS n FROM team_week_panel_nextweek_model").df()
con.execute(f"SELECT COUNT(*) AS n FROM {MODEL_VIEW}").df()